<a href="https://colab.research.google.com/github/senasung37/NLP/blob/main/word2vec_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

코드 출처1: https://wikidocs.net/50739<br>
코드 출처2: https://simonezz.tistory.com/37

#라이브러리, 데이터 준비 및 전처리

In [ ]:
#라이브러리
import re
import urllib.request
import zipfile
from lxml import etree
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
#데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/09.%20Word%20Embedding/dataset/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x7fdf8d5b02d0>)

In [ ]:
#xml파일에서 <content>와 </content> 사이의 문장 불러오기
targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)
parse_text = '\n'.join(target_text.xpath('//content/text()'))

In [ ]:
parse_text[:1000]

"Here are two reasons companies fail: they only do more of the same, or they only do what's new.\nTo me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation. Both are necessary, but it can be too much of a good thing.\nConsider Facit. I'm actually old enough to remember them. Facit was a fantastic company. They were born deep in the Swedish forest, and they made the best mechanical calculators in the world. Everybody used them. And what did Facit do when the electronic calculator came along? They continued doing exactly the same. In six months, they went from maximum revenue ... and they were gone. Gone.\nTo me, the irony about the Facit story is hearing about the Facit engineers, who had bought cheap, small electronic calculators in Japan that they used to double-check their calculators.\n(Laughter)\nFacit did too much exploitation. But exploration can go wild, too.\nA few years back, I worked closely alongside a Eu

In [ ]:
#전처리 - 괄호 제거
content_text = re.sub(r'\([^)]*\)', '', parse_text)

In [ ]:
#문장 토크나이즈 (오류 발생해서 punkt 다운로드)
content_text = content_text[0:2000000]
content_text[:500]

"Here are two reasons companies fail: they only do more of the same, or they only do what's new.\nTo me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation. Both are necessary, but it can be too much of a good thing.\nConsider Facit. I'm actually old enough to remember them. Facit was a fantastic company. They were born deep in the Swedish forest, and they made the best mechanical calculators in the world. Everybody used them. A"

In [ ]:
 sent_text = sent_tokenize(content_text)

In [ ]:
sent_text[:10]

["Here are two reasons companies fail: they only do more of the same, or they only do what's new.",
 'To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation.',
 'Both are necessary, but it can be too much of a good thing.',
 'Consider Facit.',
 "I'm actually old enough to remember them.",
 'Facit was a fantastic company.',
 'They were born deep in the Swedish forest, and they made the best mechanical calculators in the world.',
 'Everybody used them.',
 'And what did Facit do when the electronic calculator came along?',
 'They continued doing exactly the same.']

In [ ]:
#한글도 되는지 테스트
sent_tokenize("오늘은 날씨가 좋다. 그래서 나는 밖으로 나갔다. 오늘은 어떤 일이 일어날까?")

['오늘은 날씨가 좋다.', '그래서 나는 밖으로 나갔다.', '오늘은 어떤 일이 일어날까?']

In [ ]:
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

In [ ]:
normalized_text[:10]

['here are two reasons companies fail they only do more of the same or they only do what s new ',
 'to me the real real solution to quality growth is figuring out the balance between two activities exploration and exploitation ',
 'both are necessary but it can be too much of a good thing ',
 'consider facit ',
 'i m actually old enough to remember them ',
 'facit was a fantastic company ',
 'they were born deep in the swedish forest and they made the best mechanical calculators in the world ',
 'everybody used them ',
 'and what did facit do when the electronic calculator came along ',
 'they continued doing exactly the same ']

In [ ]:
#단어 토크나이징
result = [word_tokenize(sentence) for sentence in normalized_text]

In [ ]:
for line in result[:10]:
    print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']
['consider', 'facit']
['i', 'm', 'actually', 'old', 'enough', 'to', 'remember', 'them']
['facit', 'was', 'a', 'fantastic', 'company']
['they', 'were', 'born', 'deep', 'in', 'the', 'swedish', 'forest', 'and', 'they', 'made', 'the', 'best', 'mechanical', 'calculators', 'in', 'the', 'world']
['everybody', 'used', 'them']
['and', 'what', 'did', 'facit', 'do', 'when', 'the', 'electronic', 'calculator', 'came', 'along']
['they', 'continued', 'doing', 'exactly', 'the', 'same']


#영문 word2vec 모델 

In [ ]:
#word2vec 모델

from gensim.models import Word2Vec
from gensim.models import KeyedVectors

#size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.
#window = 컨텍스트 윈도우 크기
#min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)
#workers = 학습을 위한 프로세스 수
#sg = 0은 CBOW, 1은 Skip-gram.

model = Word2Vec(sentences=result, size=100, window=5, min_count=5, workers=4, sg=0)

In [ ]:
#"man"과 유사한 단어들 출력
model_result = model.wv.most_similar("man")
print(model_result)

[('thinks', 0.9844326376914978), ('woman', 0.9825385808944702), ('girl', 0.9819486737251282), ('hollywood', 0.981231689453125), ('surprise', 0.9785116910934448), ('artist', 0.9771311283111572), ('guy', 0.9763175249099731), ('quote', 0.9761906862258911), ('boy', 0.9750559329986572), ('friend', 0.974511981010437)]


In [ ]:
#모델 저장 및 모델 로드
model.wv.save_word2vec_format('eng_w2v')
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") 

In [ ]:
#저장된 모델로 다시 유사한 단어 출력
model_result = loaded_model.most_similar("know")
print(model_result)

[('feel', 0.9382627010345459), ('believe', 0.9366984367370605), ('think', 0.9340047836303711), ('mean', 0.9294184446334839), ('wonder', 0.9232097864151001), ('say', 0.9168744087219238), ('surprised', 0.9129283428192139), ('tell', 0.908724308013916), ('understand', 0.8994364738464355), ('do', 0.8945654034614563)]


#한글 word2vec 모델 학습 
(네이버 영화 리뷰)

In [ ]:
!pip install konlpy
!pip install tqdm 
import konlpy
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Okt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#네이버영화 리뷰데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")

NameError: ignored

In [ ]:
#훈련데이터 df로 정의
train_data = pd.read_table('ratings.txt')
train_data[:5]

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [ ]:
#결측값 확인
print(train_data.isnull().values.any())

True


In [ ]:
#결측값 제거
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

False


In [ ]:
#리뷰 개수 확인
print(len(train_data))

199992


In [ ]:
# 정규 표현식을 통한 한글 외 문자 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [ ]:
train_data = train_data[:10000]
train_data[:5]

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,디자인을 배우는 학생으로 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업...,1
2,4655635,폴리스스토리 시리즈는 부터 뉴까지 버릴께 하나도 없음 최고,1
3,9251303,와 연기가 진짜 개쩔구나 지루할거라고 생각했는데 몰입해서 봤다 그래 이런게 진짜 영화지,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화,1


In [ ]:
# 불용어 정의
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [ ]:
# 형태소 분석기 OKT를 사용한 토큰화 작업
okt = Okt()

tokenized_data = []
for sentence in tqdm(train_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    tokenized_data.append(stopwords_removed_sentence)

100%|██████████| 10000/10000 [00:45<00:00, 218.10it/s]


In [ ]:
#Word2Vec으로 토큰화된 네이버 영화 리뷰 데이터 학습
from gensim.models import Word2Vec
model = Word2Vec(sentences = tokenized_data, size = 100, window = 5, min_count = 5, workers = 4, sg = 0)

In [ ]:
#유사한 단어 출력
print(model.wv.most_similar("연기"))

[('스토리', 0.9997537136077881), ('배우', 0.9997503757476807), ('씨', 0.9997292160987854), ('연출', 0.9997202754020691), ('연기력', 0.9997028112411499), ('특히', 0.9996919631958008), ('넘다', 0.9996894001960754), ('연', 0.9996882677078247), ('굿', 0.9996708631515503), ('영상', 0.9996631741523743)]
